In [ ]:
from networkx.algorithms.community.quality import modularity

In [27]:
import networkx as nx
# Example graph
G = nx.karate_club_graph()

In [ ]:
def get_intra_community_modularity_2(G, comm):   
    total_weight_in_graph = G.size(weight='weight')
    def curr_combination_density(G, i, j, total_weight_in_graph):
        curr_comb_weight = G[i][j]['weight'] if G.has_edge(i,j) else 0 #Aij
        Ki = G.degree(i, weight='weight') 
        Kj = G.degree(j, weight='weight')
        curr_combination_density = ((curr_comb_weight * total_weight_in_graph) - Ki * Kj) / total_weight_in_graph
        return curr_combination_density
    community_mod = sum(curr_combination_density(G, u, v, total_weight_in_graph) for u, v in G.edges(comm) if u in comm and v in comm)
    community_mod = community_mod / total_weight_in_graph
    print("Community modularity:", community_mod)
    
    return community_mod

In [2]:
from itertools import combinations

def greedy_agglomerative(G):
    # Step 1: Initialize each node as its own community
    communities = [{node} for node in G.nodes()]
    
    # Track improvement
    improved = True
    
    while improved:
        best_increase = float('-inf')
        best_merge = None

        # Step 2: Try all community pairs
        for c1, c2 in combinations(communities, 2):
            merged = c1.union(c2)

            # Compute modularity for the merged community
            mod_merged = get_intra_community_modularity_2(G, merged)

            # Compute modularity of individual communities
            mod_c1 = get_intra_community_modularity_2(G, c1)
            mod_c2 = get_intra_community_modularity_2(G, c2)

            gain = mod_merged - (mod_c1 + mod_c2)

            if gain > best_increase:
                best_increase = gain
                best_merge = (c1, c2)

        # Step 3: Merge the best pair if improvement
        if best_merge and best_increase > 0:
            c1, c2 = best_merge
            communities.remove(c1)
            communities.remove(c2)
            communities.append(c1.union(c2))
        else:
            improved = False  # No further improvement

    return communities

In [113]:
from itertools import starmap

def get_intra_community_modularity(G, comm):
  n_edges = len(G.edges())
  print("Number of edges in the graph:", n_edges)

  community_mod = sum(wt for u, v, wt in G.edges(comm, data="weight"))
  def get_random_probability(Ku, Kv, n_edges):
      """Calculate the random probability for a pair of nodes."""
      return (Ku * Kv) / (2 * n_edges)

  randomic_edges_mod = sum(
              list(starmap(get_random_probability, 
                ((G.degree(u), G.degree(v), n_edges) for u, v in G.edges(comm)))))

  sum_mod_fraction= community_mod - randomic_edges_mod
  final_com_mod = sum_mod_fraction / (2 * randomic_edges_mod)
  return final_com_mod


# print("x:", x) 

# def intra_community_edges_analysis():
    

In [126]:
comm = [{0,1,2}, {3}]
get_intra_community_modularity(G, comm)
modularity_value = modularity(G, comm)
print("Modularity value:", modularity_value)

Number of edges in the graph: 78


NetworkXError: Node {0, 1, 2} in sequence nbunch is not a valid node.

In [ ]:
print("Intra-community modularity:", get_intra_community_modularity(G, comm))

Number of edges in the graph: 78
Intra-community modularity: 2.962616822429907
